# Iterative Methods

**Task:** Consider a system of equations
$$
\begin{cases}
5x_1+x_2+x_3=10\\
2x_1+6x_2+x_3=17\\
2x_1+2x_2+7x_3=27
\end{cases}
$$

1. Verify that matrix of the system is diagonally dominant
2. Write down the system in form $\boldsymbol{x} = \mathbf{C}\boldsymbol{x}+\boldsymbol{d}$ where $\|\mathbf{C}\|<1$.
3. Estimate norm $\|\mathbf{C}\|$ and get a priori and posteriori estimate for given $\varepsilon$
4. Conduct experiments with programs implementing Jacobi and Gauss-Seidel methods. Find number of steps required to reach desired accuracy.

## Task 1.
By definition, matrix $\mathbf{A}=\{a_{i,j}\}_{i,j=1}^n$ is diagonally dominant iff:
$$
|a_{i,i}| > \sum_{j \neq i}|a_{i,j}|, \; i=1,2,\dots,n
$$

In our case we have:
$$
\mathbf{M} = \begin{bmatrix}
5 & 1 & 1 \\ 2 & 6 & 1 \\ 2 & 2 & 7
\end{bmatrix}
$$

And, as
$$
|5| > |1| + |1| = |2|, \; |6|>|2|+|1|=|3|, \; |7|>|2|+|2|=|4|,
$$

we conclude that $\mathbf{M}$ is indeed diagonally dominant.

## Task 2.
To write down such form, we use the following relations:
$$
\mathbf{C}=\mathbf{E}-\text{diag}^{-1}(\mathbf{M})\cdot\mathbf{M}, \; \boldsymbol{d}= \text{diag}^{-1}(\mathbf{M})\cdot\boldsymbol{b}
$$

given system $\mathbf{M}\boldsymbol{x}=\boldsymbol{b}$

In [10]:
import numpy as np

# Initialing matrix M and vector b
M = np.array([[5,1,1],[2,6,1],[2,2,7]])
b = np.array([10, 17, 27])

# Finding inverse of diagonal matrix
D = np.linalg.inv(np.diag(np.diag(M)))

# Finding C and d according to formulas above
C = np.identity(3) - np.matmul(D, M)
d = np.matmul(D, b)

# Prining results
print("Matrix C equals\n", C)
print("Vector d equals\n", d)

Matrix C equals
 [[ 0.         -0.2        -0.2       ]
 [-0.33333333  0.         -0.16666667]
 [-0.28571429 -0.28571429  0.        ]]
Vector d equals
 [2.         2.83333333 3.85714286]


Therefore, we have:
$$
\mathbf{C} \approx \begin{bmatrix}
0 & -0.2 & -0.2 \\
-0.34 & 0 & -0.17 \\
-0.29 & -0.29 & 0
\end{bmatrix}, \; \boldsymbol{d} \approx \begin{bmatrix}
2 \\ 2.83 \\ 3.86
\end{bmatrix}
$$

## Task 3.

In [15]:
frobenius_norm = np.linalg.norm(C, 'fro')
inf_norm = np.linalg.norm(C, np.inf)
l1_norm = np.linalg.norm(C, 1)
l2_norm = np.linalg.norm(C, 2)

print('Frobenius norm equals', frobenius_norm)
print('Infinity norm equals', inf_norm)
print('L1 norm equals', l1_norm)
print('L2 norm equals', l2_norm)

Frobenius norm equals 0.6181862138638631
Infinity norm equals 0.5714285714285714
L1 norm equals 0.6190476190476191
L2 norm equals 0.519065019898197


Thus we take $\|\mathbf{C}\| \approx 0.6$. According to a posteriori estimate, we have:
$$
\|\boldsymbol{x}^{[k+1]}-\boldsymbol{x}^{[k]}\| < \frac{\varepsilon(1-\|\mathbf{C}\|)}{\|\mathbf{C}\|}
$$

If we put $\|\mathbf{C}\|=0.6$ we obtain
$$
\|\boldsymbol{x}^{[k+1]}-\boldsymbol{x}^{[k]}\| < \frac{2}{3}\varepsilon
$$

In our experiments we will substitute $\varepsilon=10^{-5}$, that way we should stop using iterative method when norm of vectors difference becomes smaller than $6.67\cdot 10^{-6}$.

A priori estimate claims that the number of steps is:
$$
k = \left\lfloor \frac{\ln \frac{\varepsilon(1-\|\mathbf{C}\|)}{\|\boldsymbol{x}^{[1]}-\boldsymbol{x}^{[0]}\|}}{\ln \|\mathbf{C}\|} \right\rfloor
$$

In [21]:
epsilon = 10**(-5) # Initialize epsilon with a value of 10^(-5)
C_norm = l1_norm # Take L1 norm 
x0 = np.array([0, 0, 0]) # Take zero array as a starting point
x1 = np.matmul(C, x0) + d

k = np.log(epsilon*(1-C_norm)/np.linalg.norm(x1-x0, 1))/np.log(C_norm)
print("k equals", k)

k equals 30.527639475339548


Thus we see that $k \approx 31$.

## Task 4.

In [31]:
n = 3
M = [[5,1,1],
     [2,6,1],
     [2,2,7]]
b = [10,17,27]

N = 30

print('-'*20)
print('Jacobi method')
print('-'*20)

x = [0,0,0]
y = [0,0,0]

for k in range(N):
    for i in range(n):
        y[i] = (-sum([M[i][j]*x[j] for j in range(n) if j != i]) + b[i])/M[i][i]
    d = max([abs(x[i] - y[i]) for  i in range(n)]) 
    x = y.copy()
    print('Step #{}: {}'.format(k+1, x))
    if d < epsilon:
        print('We break on step #{} with a distance of {}'.format(k+1, d))
        break

print('-'*20)
print('Gauss-Seidel method')    
print('-'*20)

x = [0,0,0] 

for k in range(N):
    d = 0
    for i in range(n):
        y = (-sum([M[i][j]*x[j] for j in range(n) if j != i]) + b[i])/M[i][i]
        d = max(d,abs(x[i] - y))
        x[i] = y
    print('Step #{}: {}'.format(k+1, x))    
    if d < epsilon:
        print('We break on step #{} with a distance of {}'.format(k+1, d))
        break

--------------------
Jacobi method
--------------------
Step #1: [2.0, 2.8333333333333335, 3.857142857142857]
Step #2: [0.6619047619047619, 1.5238095238095237, 2.476190476190476]
Step #3: [1.2, 2.1999999999999997, 3.23265306122449]
Step #4: [0.9134693877551021, 1.8945578231292517, 2.885714285714286]
Step #5: [1.0439455782312925, 2.047891156462585, 3.054849368318756]
Step #6: [0.9794518950437319, 1.976209912536443, 2.9737609329446064]
Step #7: [1.0100058309037903, 2.0112225461613216, 3.0126680549770932]
Step #8: [0.9952218797723169, 1.9945533805358877, 2.9939347494099677]
Step #9: [1.002302374010829, 2.0026035818408996, 3.002921354197656]
Step #10: [0.9988950127922889, 1.9987456496301144, 2.9985982983280772]
Step #11: [1.0005312104083617, 2.0006019460145574, 3.0006740964507417]
Step #12: [0.9997447915069401, 1.9997105804554225, 2.999676241022023]
Step #13: [1.000122635704511, 2.0001390293273498, 3.0001556080107536]
Step #14: [0.9999410725323793, 1.9999331867633707, 2.9999252385623256]
S

Thus we see that for a Jacobi method we require 19 steps and for a Gauss-Seidel method 7 steps while priori estimate is 31 steps.

Now let us try to set higher dimensionality

In [34]:
n = 7
M = [[5,1,1,7,5,7,3],
     [2,78,1,9,2,6,9],
     [3,2,23,2,8,4,4],
     [3,10,7,57,8,4,3],
     [2,5,7,2,56,4,2],
     [1,2,0,2,6,46,4],
     [2,6,13,6,8,4,46]]
b = [10,17,27,7,1,7,6]

N = 70

print('-'*20)
print('Jacobi method')
print('-'*20)

x = [0,0,0,0,0,0,0]
y = [0,0,0,0,0,0,0]

for k in range(N):
    for i in range(n):
        y[i] = (-sum([M[i][j]*x[j] for j in range(n) if j != i]) + b[i])/M[i][i]
    d = max([abs(x[i] - y[i]) for  i in range(n)]) 
    x = y.copy()
    print('Step #{}: {}'.format(k+1, x))
    if d < epsilon:
        print('We break on step #{} with a distance of {}'.format(k+1, d))
        break

print('-'*20)
print('Gauss-Seidel method')    
print('-'*20)

x = [0,0,0,0,0,0,0] 

for k in range(N):
    d = 0
    for i in range(n):
        y = (-sum([M[i][j]*x[j] for j in range(n) if j != i]) + b[i])/M[i][i]
        d = max(d,abs(x[i] - y))
        x[i] = y
    print('Step #{}: {}'.format(k+1, x))    
    if d < epsilon:
        print('We break on step #{} with a distance of {}'.format(k+1, d))
        break

--------------------
Jacobi method
--------------------
Step #1: [2.0, 0.21794871794871795, 1.173913043478261, 0.12280701754385964, 0.017857142857142856, 0.15217391304347827, 0.13043478260869565]
Step #2: [1.2405363324699707, 0.1102327306560716, 0.8280520262412611, -0.18490764257354642, -0.23968418118875784, 0.08020883785733054, -0.34906427767834597]
Step #3: [2.408044123019001, 0.23711202469640205, 1.148723305879094, -0.017131044851643466, -0.1264552145149047, 0.1900690367219763, -0.11306655296719015]
Step #4: [1.6750148915616534, 0.14512132098910716, 0.8712842370588509, -0.1762412373516101, -0.2418320082486412, 0.1165867699184711, -0.32213068548485746]
Step #5: [2.3153455623363586, 0.21856636772221164, 1.0780011689693356, -0.055097307038915286, -0.1543612371693169, 0.176668255056405, -0.15264557348691535]
Step #6: [1.8164357466986711, 0.15909880224475165, 0.9072097549000395, -0.15248342016538388, -0.22429854295076934, 0.12814056594044806, -0.28472399583141184]
Step #7: [2.21595122493

As we can clearly see, now we need around 54 steps for a Jacobi method and only 7 steps for Gauss-Seidel method which makes difference between two methods even more drastic. 